In [1]:
!pip install -q sentence-transformers datasets faiss-cpu torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 107.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install torch

In [3]:
from sentence_transformers import SentenceTransformer, InputExample, losses, models, util, LoggingHandler
from torch.utils.data import DataLoader
from datetime import datetime
import torch, torch.nn.functional as F
import os, logging, tarfile, pickle
import faiss
from datetime import datetime
import time


In [5]:
model_name = 'distilbert-base-uncased'
max_seq_length = 300
batch_size = 8
epochs = 2
pooling = "mean"
data_folder = "msmarco-data"
max_records = 10000  #

In [6]:
word_embedding_model = models.Transformer(model_name, max_seq_length=max_seq_length)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(), pooling)
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [7]:
corpus = {}
collection_filepath = os.path.join(data_folder, "collection.tsv")
if not os.path.exists(collection_filepath):
    tar_path = os.path.join(data_folder, "collection.tar.gz")
    util.http_get("https://msmarco.z22.web.core.windows.net/msmarcoranking/collection.tar.gz", tar_path)
    with tarfile.open(tar_path, "r:gz") as tar:
        tar.extractall(path=data_folder)
with open(collection_filepath, encoding="utf8") as fIn:
    for line in fIn:
        pid, passage = line.strip().split("\t")
        corpus[int(pid)] = passage

  0%|          | 0.00/1.04G [00:00<?, ?B/s]

In [8]:
queries = {}
queries_filepath = os.path.join(data_folder, "queries.train.tsv")
if not os.path.exists(queries_filepath):
    tar_path = os.path.join(data_folder, "queries.tar.gz")
    util.http_get("https://msmarco.z22.web.core.windows.net/msmarcoranking/queries.tar.gz", tar_path)
    with tarfile.open(tar_path, "r:gz") as tar:
        tar.extractall(path=data_folder)
with open(queries_filepath, encoding="utf8") as fIn:
    for line in fIn:
        qid, query = line.strip().split("\t")
        queries[int(qid)] = query

  0%|          | 0.00/18.9M [00:00<?, ?B/s]

In [9]:
qrels_url = "https://msmarco.z22.web.core.windows.net/msmarcoranking/qrels.train.tsv"
qrels_path = os.path.join(data_folder, "qrels.train.tsv")
if not os.path.exists(qrels_path):
    util.http_get(qrels_url, qrels_path)

  0%|          | 0.00/10.6M [00:00<?, ?B/s]

In [13]:
!pip install onnxruntime onnxruntime-tools torch onnx


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 107.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.7/212.7 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 9.8 MB/s eta 0:00:00


In [ ]:
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer
from onnxruntime.quantization import quantize_dynamic, QuantType
import torch, os, onnxruntime

trained_model_path ="model_path"
onnx_dir = "onnx_quantized"
os.makedirs(onnx_dir, exist_ok=True)

original_model_name = "distilbert-base-uncased"

model = SentenceTransformer(trained_model_path)
transformer_model = model._first_module().auto_model
tokenizer = AutoTokenizer.from_pretrained(original_model_name)

transformer_model.eval()

sample = "The ONNX Quant."
inputs = tokenizer(sample, return_tensors="pt", padding="max_length", max_length=300, truncation=True)
inputs = {k: v.cpu() for k, v in inputs.items()}
transformer_model = transformer_model.cpu()



torch.onnx.export(
    transformer_model,
    (inputs["input_ids"], inputs["attention_mask"]),
    f"{onnx_dir}/model.onnx",
    input_names=["input_ids", "attention_mask"],
    output_names=["last_hidden_state"],
    dynamic_axes={"input_ids": {0: "batch_size"}, "attention_mask": {0: "batch_size"}},
    opset_version=17
)

quantize_dynamic(
    model_input=f"{onnx_dir}/model.onnx",
    model_output=f"{onnx_dir}/quantized_model.onnx",
    weight_type=QuantType.QInt8
)



In [ ]:
import faiss, pickle, numpy as np, time
import onnxruntime
from transformers import AutoTokenizer

onnx_model_path = "onnx_quantized/quantized_model.onnx"
ort_session = onnxruntime.InferenceSession(onnx_model_path)
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def quantized_encode(texts, tokenizer, ort_session, max_length=300):
    all_embeddings = []
    for text in texts:
        tokens = tokenizer(text, return_tensors="np", padding="max_length", truncation=True, max_length=max_length)
        ort_inputs = {"input_ids": tokens["input_ids"], "attention_mask": tokens["attention_mask"]}
        outputs = ort_session.run(["last_hidden_state"], ort_inputs)
        last_hidden = outputs[0]
        mask = np.expand_dims(tokens["attention_mask"], -1).astype(np.float32)
        pooled = (last_hidden * mask).sum(1) / np.clip(mask.sum(1), 1e-9, None)
        all_embeddings.append(pooled[0])
    return np.vstack(all_embeddings).astype("float32")

unique_passages = list({ex.texts[1] for ex in test_examples})[:20]
chunk_embeddings = quantized_encode(unique_passages, tokenizer, ort_session)
faiss.normalize_L2(chunk_embeddings)
index = faiss.IndexFlatIP(chunk_embeddings.shape[1])
index.add(chunk_embeddings)
faiss.write_index(index, "faiss_index_chunks.index")

with open("faiss_passages.pkl", "wb") as f:
    pickle.dump(unique_passages, f)

print(" FAISS index + passages saved from quantized model.")


index = faiss.read_index("faiss_index_chunks.index")
with open("faiss_passages.pkl", "rb") as f:
    passages = pickle.load(f)

query = "What is the role of mitochondria?"
query_embedding = quantized_encode([query], tokenizer, ort_session)
faiss.normalize_L2(query_embedding)
top_k = 3
start_time = time.perf_counter()
D, I = index.search(query_embedding, top_k)
elapsed_time = time.perf_counter() - start_time

print(f"\n Inference time: {elapsed_time:.4f} seconds")
print(f"\nTop {top_k} passages for query: '{query}'\n")
for score, idx in zip(D[0], I[0]):
    print(f"Score: {score:.4f}")
    print(f"Chunk: {passages[idx]}")
    print("-" * 50)
test_examples = train_examples[-100:]
queries = [ex.texts[0] for ex in test_examples]
positives = [ex.texts[1] for ex in test_examples]

query_embeddings = quantized_encode(queries, tokenizer, ort_session)
positive_embeddings = quantized_encode(positives, tokenizer, ort_session)
faiss.normalize_L2(query_embeddings)
faiss.normalize_L2(positive_embeddings)
scores = np.sum(query_embeddings * positive_embeddings, axis=1)
average_score = scores.mean()
print(f"\nn Average cosine similarity on 100 quantized pairs: {average_score:.4f}")

sorted_idx = np.argsort(scores)[::-1]

print("\nTop 5 pairs:")
for i in sorted_idx[:5]:
    print(f"Score: {scores[i]:.4f}")
    print(f"Query: {queries[i]}")
    print(f"Passage: {positives[i]}")
    print("-" * 40)
print("\nBottom 5 pairs:")
for i in sorted_idx[-5:]:
    print(f"Score: {scores[i]:.4f}")
    print(f"Query: {queries[i]}")
    print(f"Passage: {positives[i]}")
    print("-" * 40)


In [ ]:
!pip install -q onnx onnxruntime onnxruntime-tools transformers sentence-transformers faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 76.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.7/212.7 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.2 MB/s eta 0:00:00


In [ ]:
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer
from onnxruntime.quantization import quantize_dynamic, QuantType
import torch, os, onnxruntime

trained_model_path = "output/bi-encoder-30-msmarco-2025-04-21_22-45-28"
onnx_dir = "onnx_quantized"
os.makedirs(onnx_dir, exist_ok=True)

original_model_name = "distilbert-base-uncased"

model = SentenceTransformer(trained_model_path)
transformer_model = model._first_module().auto_model
tokenizer = AutoTokenizer.from_pretrained(original_model_name)

transformer_model.eval()

sample = "This is a sample for ONNX export."
inputs = tokenizer(sample, return_tensors="pt", padding="max_length", max_length=300, truncation=True)

torch.onnx.export(
    transformer_model,
    (inputs["input_ids"], inputs["attention_mask"]),
    f"{onnx_dir}/model.onnx",
    input_names=["input_ids", "attention_mask"],
    output_names=["last_hidden_state"],
    dynamic_axes={"input_ids": {0: "batch_size"}, "attention_mask": {0: "batch_size"}},
    opset_version=13
)

quantize_dynamic(
    model_input=f"{onnx_dir}/model.onnx",
    model_output=f"{onnx_dir}/quantized_model.onnx",
    weight_type=QuantType.QInt8
)


In [ ]:
unique_passages = list({ex.texts[1] for ex in test_examples})
chunk_embeddings = model.encode(unique_passages, convert_to_tensor=True, show_progress_bar=True)
chunk_embeddings_np = chunk_embeddings.cpu().detach().numpy().astype("float32")
faiss.normalize_L2(chunk_embeddings_np)
index = faiss.IndexFlatIP(chunk_embeddings_np.shape[1])
index.add(chunk_embeddings_np)
faiss.write_index(index, "faiss_index_chunks.index")
with open("faiss_passages.pkl", "wb") as f:
    pickle.dump(unique_passages, f)

print(" FAISS index and passage text mapping saved.")

In [ ]:
index = faiss.read_index("faiss_index_chunks.index")
with open("faiss_passages.pkl", "rb") as f:
    passages = pickle.load(f)

In [ ]:
start_time = time.perf_counter()

query = "What is the role of mitochondria?"
query_embedding = model.encode(query, convert_to_numpy=True).astype("float32")
faiss.normalize_L2(query_embedding.reshape(1, -1))

top_k = 3
D, I = index.search(query_embedding.reshape(1, -1), top_k)

end_time = time.perf_counter()
elapsed_time = end_time - start_time

print(f"\nInference time: {elapsed_time:.4f} seconds")
print(f"\nTop {top_k} passages for query: '{query}'\n")
for score, idx in zip(D[0], I[0]):
    print(f"Score: {score:.4f}")
    print(f"Chunk: {passages[idx]}")
    print("-" * 50)


In [ ]:
import faiss
import pickle
import numpy as np
from tqdm import tqdm

index = faiss.read_index("faiss_index_chunks.index")
with open("faiss_passages.pkl", "rb") as f:
    faiss_passages = pickle.load(f)


test_examples = train_examples[-100:]
queries = [ex.texts[0] for ex in test_examples]
positives = [ex.texts[1] for ex in test_examples]

query_embeddings = model.encode(queries, convert_to_numpy=True, show_progress_bar=True).astype("float32")
faiss.normalize_L2(query_embeddings)

positive_embeddings = model.encode(positives, convert_to_numpy=True, show_progress_bar=True).astype("float32")
faiss.normalize_L2(positive_embeddings)
scores = np.sum(query_embeddings * positive_embeddings, axis=1)
average_score = scores.mean()
print(f"Average cosine similarity 30 test pairs (FAISS-style): {average_score:.4f}")

sorted_idx = np.argsort(scores)[::-1]

print("\nTop 5 pairs:")
for i in sorted_idx[:5]:
    print(f"Score: {scores[i]:.4f}")
    print(f"Query: {queries[i]}")
    print(f"Passage: {positives[i]}")
    print("-" * 40)

print("\nBottom 5 pairs:")
for i in sorted_idx[-5:]:
    print(f"Score: {scores[i]:.4f}")
    print(f"Query: {queries[i]}")
    print(f"Passage: {positives[i]}")
    print("-" * 40)
